In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,when,length


spark = SparkSession.builder.appName("Q1").getOrCreate()

df = spark.read.csv("emp_data.csv",inferSchema=True,header=True)

# df.count()
# df.printSchema()

"""

null_check = df.select(
    [
        count(when(col(c).isNull() | isnan(c) | (col(c) == ""), c)).alias(c)
        for c in df.columns
    ]
)

null_check.show()

"""

df = df.fillna({"LastName":"Unknown"})
df = df.filter(col("EmpID").isNotNull() & col("StartDate").isNotNull())


df = df.withColumn(
    "Current Employee Rating",
    when(col("Current Employee Rating") < 1, 1)
    .when(col("Current Employee Rating") > 5, 5)
    .otherwise(col("Current Employee Rating")),
)

# df.select("LocationCode").distinct().show()

"""
count_4_digits = df.filter(length(col("LocationCode")) == 4).count()

count_5_digits = df.filter(length(col("LocationCode")) == 5).count()
"""
 

# newdf = df.dropDuplicates(["EmpID"])
# newdf = df.dropDuplicates()



# Q4

newdf = df.groupBy("DepartmentType","Title").count()

newdf.show()


# Q5


highest_performance_df = df.groupBy("DepartmentType").agg(max("Current Employee Rating").alias("Current Employee Rating"))

highest_df = df.join(
    highest_performance_df,
    (df["DepartmentType"] == highest_performance_df["DepartmentType"]) & 
    (df["Current Employee Rating"] == highest_performance_df["Current Employee Rating"]),
    "inner"
)

highest_df.show()


spark.stop()

+-----------------+--------------------+-----+
|   DepartmentType|               Title|count|
+-----------------+--------------------+-----+
|            IT/IS|       Data Analyst |    1|
|Production       |Production Techni...|    1|
|            IT/IS|Principal Data Ar...|    8|
|            IT/IS|        Data Analyst|   10|
|Production       |    Network Engineer|    1|
|            IT/IS|Sr. Network Engineer|   30|
| Executive Office|    Network Engineer|   18|
|            IT/IS|    Network Engineer|   22|
|            IT/IS|      Data Architect|    2|
|            IT/IS|  Area Sales Manager|    7|
|            Sales|  Area Sales Manager|  119|
|            IT/IS|Enterprise Architect|    5|
|            IT/IS|          IT Support|   44|
|            Sales| Area Sales Manager?|    1|
|            Sales| Area Sales Manager.|    1|
|            IT/IS|Software Engineer...|    1|
|            IT/IS|      Sr. Accountant|    4|
|            IT/IS|             Sr. DBA|    6|
|            